In [34]:
!pip install -U langchain-ollama langchain-openai

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
!ls /content/drive/MyDrive/review_dataset

w_review_train.csv  w_review_train.parquet


In [37]:
import duckdb

con = duckdb.connect(database=':memory:', read_only=False)

In [38]:
df = con.execute("SELECT * FROM read_csv_auto('/content/drive/MyDrive/review_dataset/w_review_train.csv', parallel=false, encoding='UTF-8', ignore_errors=true)").fetchdf()

df.head()

column0  column1
0  ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...        3
1  อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...        4
2  ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...        3
3  รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...        5
4  อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...        5

In [39]:
df.shape

(40000, 2)

In [40]:
con.execute("COPY df TO '/content/drive/MyDrive/review_dataset/w_review_train.parquet' (FORMAT PARQUET)")

In [41]:
parquet_avg_rating_direct = con.execute("SELECT AVG(column1) FROM '/content/drive/MyDrive/review_dataset/w_review_train.parquet' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [42]:
parquet_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.parquet'

In [43]:
parquet_avg_rating_direct = con.execute(f"SELECT AVG(column1) FROM '{parquet_file_path}' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [44]:
coffee_keywords = [
    'กาแฟ', 'คาเฟ่', 'coffee', 'cafe', 'เอสเปรสโซ่', 'ลาเต้',
    'มอคค่า', 'คาปูชิโน่', 'เฟรนช์เพรส', 'cold brew', 'drip',
    'espresso', 'latte', 'americano', 'macchiato', 'brew',
    'specialty', 'slow bar', 'ร้านกาแฟ', 'ร้านคาเฟ่', 'กาแฟสด',
    'เมนูกาแฟ', 'กาแฟนม', 'กาแฟดำ', 'ร้านบาริสต้า', 'barista', 'coffee shop'
]

where_clause = " OR ".join([f"column0 LIKE '%{keyword}%'" for keyword in coffee_keywords])

In [45]:
%%time
coffee_reviews_df = con.execute(f"SELECT * FROM '{parquet_file_path}' WHERE {where_clause}").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 2.11 s, sys: 5.33 ms, total: 2.11 s
Wall time: 3.73 s


In [46]:
csv_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.csv'

In [47]:
%%time
coffee_reviews_df_csv = con.execute(f"SELECT * FROM read_csv_auto('{csv_file_path}', parallel=false, encoding='UTF-8', ignore_errors=true) WHERE {where_clause}").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 4.72 s, sys: 74 ms, total: 4.79 s
Wall time: 4.91 s


In [48]:
coffee_reviews_df.head()

column0  column1
0  ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...        3
1  วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...        4
2  สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...        3
3  ร้านอาหารญี่ปุ่นร้านนี้ ใจจริงไม่อยากแนะนำเลยค...        5
4  เดือนแรกที่เค้าต่อคิวกัน 2 - 3 ชั่วโมง เราก็ว่...        5

In [49]:
coffee_reviews_df.shape

(4981, 2)

In [50]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [51]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [52]:
!ollama pull gemma3:270m

Error: ollama server not responding - could not connect to ollama server, run 'ollama serve' to start it


In [53]:
!ollama list

NAME    ID    SIZE    MODIFIED 


In [54]:
!ollama run gemma3:270m "สวัสดี"


สวัสดีค่ะ ยินดีที่ได้รู้จักค่ะ 😊




In [55]:
!curl -s http://127.0.0.1:11434/api/tags

{"models":[{"name":"gemma3:270m","model":"gemma3:270m","modified_at":"2025-10-16T10:41:38.312856418Z","size":291554930,"digest":"e7d36fb2c3b3293cfe56d55889867a064b3a2b22e98335f2e6e8a387e081d6be","details":{"parent_model":"","format":"gguf","family":"gemma3","families":["gemma3"],"parameter_size":"268.10M","quantization_level":"Q8_0"}}]}

In [56]:
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [57]:
llm = OllamaLLM(model="gemma3:270m")

In [58]:
response = llm.invoke("สวัสดี")
print(response)

สวัสดีค่ะ ยินดีที่ได้รู้จักค่ะ 😊



In [59]:
template = """
คุณเป็นครูสอน Python
คำถาม : {question}
"""

In [60]:
prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

In [61]:
formatted_prompt = prompt.format(question="เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10")
print(formatted_prompt)


คุณเป็นครูสอน Python
คำถาม : เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10



In [62]:
chain = prompt | llm

In [63]:
response = chain.invoke({"question": "เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10"})

In [64]:
response

'```python\ndef find_sum_of_numbers(numbers):\n    """\n    หาผลรวมของตัวเลข 1 ถึง 10\n    """\n    total = 0\n    for number in numbers:\n        total += number\n    return total\n\n# ตัวอย่างการใช้งาน\nnumbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\nresult = find_sum_of_numbers(numbers)\nprint(result)\n```\n\nโค้ดนี้มีฟังก์ชัน `find_sum_of_numbers` ที่มีหน้าที่หลักคือการหาผลรวมของตัวเลข 1 ถึง 10 โดยใช้ `for` loop และ `sum()` function\n\n**คำอธิบายโค้ด:**\n\n1.  **`def find_sum_of_numbers(numbers):`**:  กำหนดชื่อฟังก์ชัน `find_sum_of_numbers` ที่จะทำงาน\n2.  **`total = 0`**:  เริ่มต้นค่า `total` ที่จะเก็บผลรวมของตัวเลข\n3.  **`for number in numbers:`**:  ทำการวนลูปผ่านแต่ละตัวเลขใน `numbers`\n4.  **`total += number`**:  ทำการเพิ่มค่า `number` เข้าไปใน `total`\n5.  **`return total`**:  คืนค่า `total`  ซึ่งเป็นผลรวมของตัวเลขทั้งหมด\n6.  **`numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`**:  สร้างตัวแปร `numbers` ที่มีค่า 1, 2, 3, 4, 5, 6, 7, 8, 9, 10\n7.  **`result = find_sum_of_numbers(numbers)

In [65]:
from IPython.display import display, Markdown

In [66]:
display(Markdown(response))

```python
def find_sum_of_numbers(numbers):
    """
    หาผลรวมของตัวเลข 1 ถึง 10
    """
    total = 0
    for number in numbers:
        total += number
    return total

# ตัวอย่างการใช้งาน
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = find_sum_of_numbers(numbers)
print(result)
```

โค้ดนี้มีฟังก์ชัน `find_sum_of_numbers` ที่มีหน้าที่หลักคือการหาผลรวมของตัวเลข 1 ถึง 10 โดยใช้ `for` loop และ `sum()` function

**คำอธิบายโค้ด:**

1.  **`def find_sum_of_numbers(numbers):`**:  กำหนดชื่อฟังก์ชัน `find_sum_of_numbers` ที่จะทำงาน
2.  **`total = 0`**:  เริ่มต้นค่า `total` ที่จะเก็บผลรวมของตัวเลข
3.  **`for number in numbers:`**:  ทำการวนลูปผ่านแต่ละตัวเลขใน `numbers`
4.  **`total += number`**:  ทำการเพิ่มค่า `number` เข้าไปใน `total`
5.  **`return total`**:  คืนค่า `total`  ซึ่งเป็นผลรวมของตัวเลขทั้งหมด
6.  **`numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`**:  สร้างตัวแปร `numbers` ที่มีค่า 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
7.  **`result = find_sum_of_numbers(numbers)`**:  เรียกฟังก์ชัน `find_sum_of_numbers`  เพื่อหาผลรวมของ `numbers`
8.  **`print(result)`**:  แสดงผลลัพธ์ของการหาผลรวม


In [67]:
prompt1 = PromptTemplate(
    template="แปลโจทย์เกี่ยวกับ {question}"
)

chain1 = prompt1 | llm

In [68]:
prompt2 = PromptTemplate(
    template="ทำความเข้าใจโจทย์ที่แปลแล้วต่อไปนี้ {translate}"
)

chain2 = prompt2 | llm

In [69]:
prompt3 = PromptTemplate(
    template="แก้โจทย์ตามที่เข้าใจต่อไปนี้ {understand}"
)

chain3 = prompt3 | llm

In [70]:
prompt4 = PromptTemplate(
    template="อธิบายวิธิีทำจากผลลัทธ์ต่อไปนี้ {solve}"
)

chain4 = prompt4 | llm

In [71]:
parser_chain = StrOutputParser()

In [72]:
full_chain = chain1 | chain2 | chain3 | chain4 | parser_chain

In [73]:
result = full_chain.invoke({"question":"2x + 5 = 15"})

In [74]:
result

'วิธิีทำจากผลลัทธ์ต่อไปนี้ **เป็นแนวคิดหลักที่สำคัญในการทำความเข้าใจและพัฒนาการทำงานขององค์กร** โดยมุ่งเน้นไปที่การสร้างและจัดการกระบวนการทำงานที่ซับซ้อนและมีประสิทธิภาพ โดยมีเป้าหมายหลักคือการสร้างสภาพแวดล้อมที่ทำงานร่วมกันอย่างมีประสิทธิภาพและสร้างผลลัพธ์ที่น่าเชื่อถือ\n'

In [75]:
import os
from langchain_openai import ChatOpenAI

In [76]:
OPENAI_API_KEY = "xxx"

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [77]:
llm2 = ChatOpenAI(model="gpt-4o-mini")

In [78]:
chain1 = prompt1 | llm2
chain2 = prompt2 | llm2
chain3 = prompt3 | llm2
chain4 = prompt4 | llm2

full_chain = chain1 | chain2 | chain3 | chain4 | parser_chain

In [79]:
formatted_result = result.replace('\\[', '$$') \
                         .replace('\\]', '$$') \
                         .replace('\\(', '$') \
                         .replace('\\)', '$')

In [80]:
display(Markdown(formatted_result))

วิธิีทำจากผลลัทธ์ต่อไปนี้ **เป็นแนวคิดหลักที่สำคัญในการทำความเข้าใจและพัฒนาการทำงานขององค์กร** โดยมุ่งเน้นไปที่การสร้างและจัดการกระบวนการทำงานที่ซับซ้อนและมีประสิทธิภาพ โดยมีเป้าหมายหลักคือการสร้างสภาพแวดล้อมที่ทำงานร่วมกันอย่างมีประสิทธิภาพและสร้างผลลัพธ์ที่น่าเชื่อถือ


In [81]:
prompt_template = """
[1. Role/Context บอก LLM ว่าควรทำตัวเป็นอะไร]
คุณเป็น AI ที่เชี่ยวชาญในการวิเคราะห์รีวิวภาษาไทย

[2. Task Description บอกงานที่ต้องทำ]
จาก Review ต่อไปนี้ ให้ Extract 3 keywords
ที่เป็นตัวแทนของ Review นี้ได้ดีที่สุด

[3. Input Data ข้อมูลที่ต้องประมวลผล]
Review: {review}

[4. Output Format บอกรูปแบบของคำตอบ]
ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'keywords'
เช่น {{"keywords": "กาแฟ, บรรยากาศ, ราคา"}

[5. Constraints ข้อจำกัดหรือเงื่อนไข]
- ต้องเป็นคำภาษาไทย
- คั่นด้วย comma
- หากไม่พบให้ตอบ 'ไม่ระบุ'

[6. Examples (optional) ตัวอย่าง input/output]
ตัวอย่าง
Input: "กาแฟอร่อย บรรยากาศดี ราคาแพง"
Output: {"keywords": "กาแฟ, บรรยากาศ, ราคา"}
"""

In [82]:
prompt = """
Extract keywords from this review
{review}
"""

# Pros Simple, fast
# Cons อาจได้ผลไม่ดีกับ Tasks ซับซ้อน

In [83]:
prompt = """
Extract keywords from reviews.

Example 1
Review: "กาแฟอร่อย บริการดี"
Keywords: กาแฟ, บริการ

Example 2
Review: "ราคาแพง แต่คุ้มค่า"
Keywords: ราคา, คุณภาพ

Now extract from this review
Review: {review}
Keywords:
"""

# Pros ได้ผลดีกว่า Zero-shot
# Cons Prompt ใช้ Tokens มากขึ้น

In [84]:
prompt = """
Review: {review}

ขั้นตอนการ Extract keywords
1. อ่านรีวิวและเข้าใจความหมาย
2. ระบุหัวข้อหลักที่กล่าวถึง
3. เลือก 3 keywords ที่สำคัญที่สุด
4. ตอบในรูปแบบ JSON: {{"keywords": "คำ1, คำ2, คำ3"}}

ให้ทำทีละขั้นตอน
"""

# Pros ได้ผลดีกับ Reasoning tasks
# Cons ใช้ Tokens มากกว่า ช้ากว่า

In [85]:
keyword_prompt = PromptTemplate(
template = """
จาก Review ต่อไปนี้
Extract 3 keywords ที่เป็นตัวแทนของ Review นี้

Review: {review}

ขั้นตอนการ Extract keywords
1. อ่านรีวิวและเข้าใจความหมาย
2. ระบุหัวข้อหลักที่กล่าวถึง
3. เลือก 3 keywords ที่สำคัญที่สุด
4. ตอบในรูปแบบ JSON โดยมี key คือ keyword เช่น :{{"keywords": "คำ1, คำ2, คำ3"}}

ให้ทำทีละขั้นตอน
""")

In [86]:
sample_reviews = coffee_reviews_df['column0'].head(2)
sample_reviews

0    ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...
1    วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...
Name: column0, dtype: object

In [87]:
sample_reviews.to_csv('/content/sample_reviews.csv', index=False)

In [88]:
keyword_extract = keyword_prompt | llm | StrOutputParser()

In [89]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 38.1 ms, sys: 4.19 ms, total: 42.3 ms
Wall time: 4.5 s


In [90]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text ```json
{
  "keywords": [
    "ร้านอาหารใหญ่",
    "ห้องน้ำ",
    "กาแฟ",
    "น้ำผึ้ง",
    "น้ำผึ้งมาราด",
    "แพงเวอร์",
    "อย่าสั่ง",
    "ลาบไข่ต้ม",
    "ไข่มันคาว",
    "ไม่ประทับใจ",
    "กินฟรี",
    "ขนมหวาน",
    "ขนมไทย",
    "เบิ้ล"
  ]
}
```

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text ```json
{
  "keywords": [
    "ลาเต้",
    "ร้อน",
    "แก้วนึง",
    "จิบ",
    "ฟังเพลง",
    "พักผ่อน",
    "ทำกิจกรรม",
    "พักผ่อน",
    "สบาย",
    "สนุก",
    "มีชีวิตชีวา"
  ]
}
```



In [91]:
keyword_extract = keyword_prompt | llm2 | StrOutputParser()

In [92]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text ```json
{
  "keywords": [
    "ร้านอาหารใหญ่",
    "ห้องน้ำ",
    "กาแฟ",
    "น้ำผึ้ง",
    "น้ำผึ้งมาราด",
    "แพงเวอร์",
    "อย่าสั่ง",
    "ลาบไข่ต้ม",
    "ไข่มันคาว",
    "ไม่ประทับใจ",
    "กินฟรี",
    "ขนมหวาน",
    "ขนมไทย",
    "เบิ้ล"
  ]
}
```

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text ```json
{
  "keywords": [
    "ลาเต้",
    "ร้อน",
    "แก้วนึง",
    "จิบ",
    "ฟังเพลง",
    "พักผ่อน",
    "ทำกิจกรรม",
    "พักผ่อน",
    "สบาย",
    "สนุก",
    "มีชีวิตชีวา"
  ]
}
```



In [93]:
!ollama pull scb10x/typhoon2.1-gemma3-4b

In [94]:
!ollama list

NAME                                  ID              SIZE      MODIFIED               
scb10x/typhoon2.1-gemma3-4b:latest    8cfab9097c9d    2.6 GB    Less than a second ago    
gemma3:270m                           e7d36fb2c3b3    291 MB    About a minute ago        


In [95]:
llm3 = OllamaLLM(model="gemma3:270m")

In [96]:
keyword_extract = keyword_prompt | llm3 | StrOutputParser()

In [97]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 38.6 ms, sys: 1.14 ms, total: 39.7 ms
Wall time: 3.8 s


In [98]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text ```json
{
  "keywords": [
    "ร้านอาหารใหญ่",
    "ห้องน้ำ",
    "กาแฟ",
    "น้ำผึ้ง",
    "น้ำผึ้งมาราด",
    "แพงเวอร์",
    "อย่าสั่งเลย",
    "ลาบไข่ต้ม",
    "ไข่มันคาว",
    "ประทับใจ",
    "เบิ้ล",
    "ขนมไทย",
    "ไม่ประทับใจ"
  ]
}
```

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text ```json
{
  "keywords": [
    "ลาเต้",
    "รสชาติ",
    "กลิ่น",
    "เมพพ",
    "ซากอารยธรรม",
    "ซอยไรจำไม่ได้",
    "แต่อยู่ตรงข้าม Villa"
  ]
}
```



In [99]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class ReviewKeywords(BaseModel):
    keywords: str = Field(
        ...,  # Required field
        description="3 keywords separated by comma",
        examples=["กาแฟ, บรรยากาศ, ราคา"]
    )

class KeywordsExtraction(BaseModel):
    extracted_qualities: ReviewKeywords

In [100]:
keywords_parser = JsonOutputParser(pydantic_object=KeywordsExtraction)

In [101]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [102]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 52.3 ms, sys: 2.96 ms, total: 55.2 ms
Wall time: 6.26 s


In [103]:
results[0]['text']['keywords']

['ร้านอาหารใหญ่',
 'ห้องน้ำ',
 'กาแฟ',
 'น้ำผึ้ง',
 'แพงเวอร์',
 'อย่าสั่งเลย',
 'ลาบไข่ต้ม',
 'ไข่มันคาว',
 'ทอดมันหัวปลีกรอบอร่อย',
 'เบิ้ล',
 'ขนมไทย',
 'ขนมหวาน']

In [104]:
results[1]['text']['keywords']

['ลาเต้',
 'ร้อน',
 'แก้วนึง',
 'ร้อน',
 'ไป',
 'ซดกาแฟ',
 'ร้านวาวี',
 'อารีย์',
 'Starbuck',
 'แฟนๆ Starbuck',
 'กาแฟ',
 'อร่อย',
 'ดี',
 'ราคาไม่แพง',
 'กินบ่อย',
 'จุกได้',
 'กิน',
 'ได้',
 'ทิ้ง',
 'อร่อยใจกลางเมือง']

In [105]:
llm3 = OllamaLLM(
    model="scb10x/typhoon2.1-gemma3-4b",
    temperature=0.05
)

keyword_extract = keyword_prompt | llm3 | keywords_parser

In [106]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 127 ms, sys: 7.54 ms, total: 134 ms
Wall time: 16.1 s


In [107]:
results[0]['text']['keywords'], results[1]['text']['keywords']

(['ราคาแพง', 'ทอดมันหัวปลี', 'อาหารไทย'], ['วาวี', 'ลาเต้', 'อารีย์'])

In [108]:
keyword_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract keywords 3 keywords เท่านั้น ที่เป็นตัวแทนของ Review นี้ได้ดีที่สุด

    Review: {review}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'keywords'
    และค่าเป็นข้อความที่ดึงได้คั่นด้วย comma
    เช่น {{"keywords": "กาแฟ, บรรยากาศ, ราคา"}}

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [109]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [110]:
inputs = [{"review": r} for r in sample_reviews]

In [111]:
%%time
results = keyword_extract.batch(inputs)

CPU times: user 16.6 ms, sys: 2.19 ms, total: 18.8 ms
Wall time: 2.27 s


In [112]:
results

[{'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
 {'keywords': 'กาแฟ, บรรยากาศ, รสชาติ'}]

In [113]:
sample_reviews_1000 = coffee_reviews_df['column0'].head(100).tolist()
inputs_1000 = [{"review": r} for r in sample_reviews_1000]

BATCH_SIZE = 8

results_1000 = []

In [114]:
from tqdm import tqdm

In [115]:
for i in tqdm(range(0, len(inputs_1000), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_1000[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_1000.append({"review": inp["review"], "text": out})

Extracting keywords: 100%|██████████| 13/13 [02:12<00:00, 10.18s/it]


In [116]:
for result in results_1000[0:10]:
    print(result['text']['keywords'])

ราคา, บรรยากาศ, ไข่ต้ม
กาแฟ, บรรยากาศ, รสชาติ
โดนัท, รสชาติ, ความหวาน
วาซาบิ, ปลาซาบะ, ไข่ตุ๋น
โดนัท, Original Glazed, คิว
วังพญาไท, นรสิงห์, บรรยากาศคลาสสิก
อาหารทะเล, ราคา, บรรยากาศ
กาแฟ, เค้ก, พนักงาน
Shibuya Honey Toast, ฮันนี่ โทสท์, ราคา
คาเฟ่, วานิลลา, คาโบนาร่า


In [117]:
  results_1000[0]

{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
 'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'}}

In [118]:
context_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract context ใน Review ที่เป็นตัวแทนที่บรรยายคำว่า {keyword} ได้ดีที่สุด

    Review: {review}

    ตอบผลลัพธ์ในรูปแบบ JSON โดยมีคีย์เป็น 'context'
    และค่าเป็นข้อความที่ดึงได้จาก Review

    ตัวอย่าง
    keyword คือ "กาแฟ"
    context: "กาแฟที่นี่อร่อยมาก หอมกลิ่นกาแฟคั่วสด"

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [119]:
sentiment_prompt = PromptTemplate(
    template="""จากข้อความต่อไปนี้ ให้วิเคราะห์ว่าเป็น sentiment แบบไหน
    เลือกจากตัวเลือกเหล่านี้เท่านั้น: "positive", "negative", "neutral"

    ข้อความ: {message}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'sentiment'
    และค่าเป็นหนึ่งใน list ต่อไปนี้ [positive, negative, neutral]

    คำแนะนำ
    - positive คือ ข้อความที่แสดงความพอใจ ชอบ ดี
    - negative คือ ข้อความที่แสดงความไม่พอใจ ไม่ชอบ แย่
    - neutral คือ ข้อความที่ไม่แสดงความรู้สึกชัดเจน เป็นกลาง

    # ตัวอย่าง
    # ข้อความ "อร่อยมาก" → {{"sentiment": "positive"}}
    # ข้อความ "แพงไป" → {{"sentiment": "negative"}}
    # ข้อความ "ปกติ" → {{"sentiment": "neutral"}}
    """
)


In [120]:
class ContextKeyword(BaseModel):
    context: str = Field(..., description="context that best represent this keyword")

class ContexExtraction(BaseModel):
    extracted: ContextKeyword

context_parser = JsonOutputParser(pydantic_object=ContexExtraction)

In [121]:
context_extract = context_prompt | llm3 | context_parser

In [122]:
class Sentiment(BaseModel):
    sentiment: str = Field(..., description="sentiment of review")

class SentimentExtraction(BaseModel):
    extracted: Sentiment


sentiment_parser = JsonOutputParser(pydantic_object=SentimentExtraction)

In [123]:
sentiment_extract = sentiment_prompt | llm3 | sentiment_parser

In [124]:
keywords = results_1000[0]['text']['keywords']


In [125]:
context_inputs = []

for review_data in tqdm(results_1000, desc="Preparing context inputs"):
    review = review_data['review']
    keywords = review_data['text']['keywords']
    for keyword in keywords:
        context_inputs.append({"review": review, "keyword": keyword.strip()})

Preparing context inputs: 100%|██████████| 100/100 [00:00<00:00, 78618.63it/s]


In [126]:
len(context_inputs)

2521

In [127]:
context_results = []
for i in tqdm(range(len(results_1000)), desc="Extracting contexts"):
   result = context_extract.batch(context_inputs[i*3:(i*3)+3])
   contexts_list = [d['context'] for d in result]
   results_1000[i]['contexts'] = contexts_list

Extracting contexts: 100%|██████████| 100/100 [09:04<00:00,  5.45s/it]


In [128]:
results_1000[:2]

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
  'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
  'contexts': ['ร้านอาหารใหญ่มากกกกกกก',
   'ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี',
   'ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย']},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสว

In [129]:
sentiment_inputs = []

for review_data in tqdm(results_1000, desc="Preparing sentiment inputs"):
    contexts = review_data['contexts']
    for context in contexts:
        sentiment_inputs.append({"message": context})

Preparing sentiment inputs: 100%|██████████| 100/100 [00:00<00:00, 315598.50it/s]


In [130]:
len(sentiment_inputs)

300

In [131]:
sentiment_inputs[:6]

[{'message': 'ร้านอาหารใหญ่มากกกกกกก'},
 {'message': 'ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี'},
 {'message': 'ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย'},
 {'message': 'ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี'},
 {'message': 'ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว'},
 {'message': 'ทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล'}]

In [132]:
for i in tqdm(range(len(results_1000)), desc="Extracting sentiment"):
   result = sentiment_extract.batch(sentiment_inputs[i*3:(i*3)+3])
   sentiment_list = [d['sentiment'] for d in result]
   results_1000[i]['sentiment'] = sentiment_list

Extracting sentiment: 100%|██████████| 100/100 [04:19<00:00,  2.59s/it]


In [133]:
results_1000[0:2]

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
  'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
  'contexts': ['ร้านอาหารใหญ่มากกกกกกก',
   'ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี',
   'ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย'],
  'sentiment': ['neutral', 'neutral', 'negative']},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรยา

In [134]:
import pandas as pd

df = pd.DataFrame([
    {
        'review': r['review'],
        **{f'keyword{i+1}': k.strip() for i, k in enumerate(r['text']['keywords'])},
        **{f'context{i+1}_keyword': c for i, c in enumerate(r['contexts'])},
        **{f'sentiment{i+1}': s for i, s in enumerate(r['sentiment'])}
    }
    for r in results_1000
])

In [135]:
df.to_csv('results.csv', index=False, encoding='utf-8-sig')